import pandas as pd
from tqdm import tqdm
from utils import get_embedding
import torch
import pickle
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
fn = "thucnews/thucnews.csv"
df = pd.read_csv(fn)
df.columns

Index(['index', 'label', 'title', 'content', 'entity', 'event',
       'event_sentence', 'top_ten', 'train_test', 'keyword'],
      dtype='object')

In [3]:
events = df['top_ten']
events.shape

(3163,)

In [4]:
def get_each_event(event_str):
    event_list = event_str.split("#")
    return event_list

In [5]:
embedding = get_embedding()
def embed_event(x):
    x_embeddings = torch.zeros(10, 768)
    i = 0
    for item in x:
        event_embedding = torch.tensor(embedding.embed_query(item), dtype=float)
        x_embeddings[i, :] = event_embedding
        i = i + 1
    return x_embeddings

d:\app\Anaconda3\envs\embed\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


获取的嵌入模型是: text2vec-base


In [6]:
with torch.no_grad():
    evt_embedding_list = []
    for item in tqdm(events):
        event_list = get_each_event(item)
        evt_embedding_list.append(embed_event(event_list))
    print(len(evt_embedding_list))

100%|██████████| 3163/3163 [22:26<00:00,  2.35it/s]

3163


In [7]:
print(len(evt_embedding_list))

3163


In [8]:
evt_embedding_list[0].shape

torch.Size([10, 768])

In [10]:
with open("thucnews/views/pkl/event_embeddings.pickle", "wb") as file:
    pickle.dump(evt_embedding_list, file=file)

In [4]:
with open("thucnews/views/pkl/event_embeddings.pickle", "rb") as file:
    evt_embedding_list = pickle.load(file=file)

In [5]:
len(evt_embedding_list)

3163

In [20]:
s = cosine_similarity(evt_embedding_list[33], evt_embedding_list[37])